# Import required libraries

In [26]:
from google.colab import files
from google.colab.patches import cv2_imshow
from IPython.display import Video
import argparse
import cv2
import sys
import numpy as np

# All the funtions

In [27]:
# Helper function to check if the current time is between the given bounds
def between(cap, lower: int, upper: int) -> bool:
    return lower <= int(cap.get(cv2.CAP_PROP_POS_MSEC)) < upper


In [28]:
# Gaussian kernel generator for Filterring
def gaussian_kernel(size: int, sigma: float) -> np.ndarray:
    # Create an empty kernel
    kernel = np.zeros((size, size), np.float32)

    # Calculate the kernel values
    for x in range(size):
        for y in range(size):
            kernel[x, y] = (1 / (2 * np.pi * sigma ** 2)) * np.exp(
                -((x - (size - 1) / 2) ** 2 + (y - (size - 1) / 2) ** 2) / (2 * sigma ** 2)
            )

    # Normalize the kernel so that the sum of all values is 1
    return kernel / np.sum(kernel)

In [29]:
#ID card detection
def id_card_detection(frame):
    # Step 1: Convert the frame to grayscale
    gray_image = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Step 2: Apply Histogram Equalization
    equalized_image = cv2.equalizeHist(gray_image)

    # Step 3: Apply binary thresholding to isolate white areas
    _, binary = cv2.threshold(equalized_image, 200, 255, cv2.THRESH_BINARY)

    # Step 4: Use morphological operations (dilation and erosion)
    kernel = np.ones((5, 5), np.uint8)
    morph = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)

    # Step 5: Find contours
    contours, _ = cv2.findContours(morph, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Step 6: Draw rectangles around detected contours and find the largest ID card region
    largest_contour = None
    largest_area = 0

    for contour in contours:
        # Get the bounding box of each contour
        x, y, w, h = cv2.boundingRect(contour)

        # Filter based on aspect ratio or size to ensure it's the ID card
        aspect_ratio = w / float(h)
        if 1.5 < aspect_ratio < 3.0:  # Adjust these values based on your ID card's expected shape
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            # Calculate the area of the contour
            area = w * h

            # Check if this is the largest area found
            if area > largest_area:
                largest_area = area
                largest_contour = contour

    # Step 7: If a largest contour is found, extract the ID card region
    id_card_region = None
    if largest_contour is not None:
        x, y, w, h = cv2.boundingRect(largest_contour)
        id_card_region = frame[y:y + h, x:x + w]

    return frame, id_card_region

In [30]:
# Temple matching function
def match_template(id_card_region, resized_template):
    # Ensure both images are in the correct format
    id_card_region = cv2.convertScaleAbs(id_card_region)
    resized_template = cv2.convertScaleAbs(resized_template)

    # Convert only the resized template to grayscale for matching
    if len(resized_template.shape) == 3:
        resized_template_gray = cv2.cvtColor(resized_template, cv2.COLOR_BGR2GRAY)
    else:
        resized_template_gray = resized_template  # Already grayscale

    # Check sizes before matching
    id_card_height, id_card_width = id_card_region.shape[:2]
    template_height, template_width = resized_template_gray.shape

    if id_card_height < template_height or id_card_width < template_width:
        print("Error: Template size is larger than the ID card region size.")
        return id_card_region  # Return the original id_card_region without matching

    # Perform template matching using Normalized Cross-Correlation
    method = cv2.TM_CCOEFF_NORMED
    result = cv2.matchTemplate(cv2.cvtColor(id_card_region, cv2.COLOR_BGR2GRAY), resized_template_gray, method)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)

    # Get the location of the best match
    h, w = template_height, template_width
    location = max_loc
    bottom_right = (location[0] + w, location[1] + h)

    # Draw a rectangle around the matched region on the original color image
    cv2.rectangle(id_card_region, location, bottom_right, (0, 255, 0), 2)

    return id_card_region


In [31]:
# Draw function for optical flow
def draw_optical_flow_arrows(frame, flow, step=16):
    h, w = frame.shape[:2]
    y, x = np.mgrid[step//2:h:step, step//2:w:step].astype(int)
    # Extract the flow vectors
    fx, fy = flow[y, x].T
    frame_with_arrows = frame.copy()

    # Draw the arrows on the frame
    for (x1, y1, dx, dy) in zip(x.flatten(), y.flatten(), fx.flatten(), fy.flatten()):
        # Calculate the endpoint of the arrow
        x2 = int(x1 + dx)
        y2 = int(y1 + dy)
        cv2.arrowedLine(frame_with_arrows, (x1, y1), (x2, y2), (255, 0, 0), 2, tipLength=0.5)
    return frame_with_arrows

In [32]:
## Most image processing operations code here
def apply_processing(cap, frame, lower, upper, process_type, templates):
    """Applies different image processing techniques based on the section of the video."""
    processed_frame = frame.copy()  # Create a copy of the frame for processing

    if between(cap, lower, upper):
        if process_type == "blur":
            # Apply Gaussian Blur
            processed_frame = cv2.GaussianBlur(frame, (7, 7), 0)
            cv2.putText(processed_frame, 'Smoothing using Gaussian Blur', (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, .5, (0, 0, 255), 2)

        elif process_type == "sharpen":
            # Apply sharpening
            sharpen_kernel = np.array([[-1, -1, -1],
                                       [-1, 9, -1],
                                       [-1, -1, -1]])
            processed_frame = cv2.filter2D(frame, -1, sharpen_kernel)
            cv2.putText(processed_frame, 'Sharpening', (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, .5, (0, 0, 255), 2)

        elif process_type == "sobel":
            # Apply Sobel edge detection
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
            sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
            sobel_combined = cv2.magnitude(sobelx, sobely)
            processed_frame = cv2.cvtColor(sobel_combined.astype(np.uint8), cv2.COLOR_GRAY2BGR)
            cv2.putText(processed_frame, 'Sobel Edge Detection', (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, .5, (0, 0, 255), 2)

        elif process_type == "canny":
            # Apply Canny edge detection
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            canny_edges = cv2.Canny(gray, 100, 200)
            processed_frame = cv2.cvtColor(canny_edges, cv2.COLOR_GRAY2BGR)
            cv2.putText(processed_frame, 'Canny Edge Detection (Thresholds: 100, 200)', (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, .5, (0, 0, 255), 2)

        elif process_type == "dft":
            # Show DFT spectrum
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            dft = cv2.dft(np.float32(gray), flags=cv2.DFT_COMPLEX_OUTPUT)
            dft_shift = np.fft.fftshift(dft)
            magnitude = cv2.magnitude(dft_shift[:, :, 0], dft_shift[:, :, 1])

            # Normalize the magnitude to the range [0, 255]
            magnitude = np.log1p(magnitude)  # Use log scale for better visualization
            magnitude = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
            magnitude = np.uint8(magnitude)

            # Convert back to BGR for displaying
            magnitude_bgr = cv2.cvtColor(magnitude, cv2.COLOR_GRAY2BGR)
            cv2.putText(magnitude_bgr, 'DFT Magnitude Spectrum', (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, .5, (0, 0, 255), 2)

            processed_frame = magnitude_bgr  # Set the processed frame to the DFT display

        elif process_type == "fourier_low":
            # Apply Gaussian Low Pass Filter
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            kernel_size = 15
            sigma = 15
            kernel = gaussian_kernel(kernel_size, sigma)

            processed_frame = cv2.filter2D(gray, -1, kernel)
            processed_frame = cv2.normalize(processed_frame, None, 0, 255, cv2.NORM_MINMAX)
            processed_frame = np.uint8(processed_frame)
            processed_frame = cv2.cvtColor(processed_frame, cv2.COLOR_GRAY2BGR)

            cv2.putText(processed_frame, 'Gaussian Low Pass Filter', (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, .5, (0, 0, 255), 2)

        elif process_type == "fourier_high":
            # Apply Gaussian High Pass Filter
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            # Create a Gaussian kernel for low-pass filtering
            kernel_size = 9
            sigma = 9
            low_pass_kernel = gaussian_kernel(kernel_size, sigma)

            # Create a high-pass kernel by subtracting low-pass kernel from an identity kernel
            high_pass_kernel = np.zeros((kernel_size, kernel_size), np.float32)
            high_pass_kernel[kernel_size // 2, kernel_size // 2] = 1
            high_pass_kernel -= low_pass_kernel

            # Apply the high-pass filter using convolution
            processed_frame = cv2.filter2D(gray, -1, high_pass_kernel)

            # Normalize the result
            processed_frame = cv2.normalize(processed_frame, None, 0, 255, cv2.NORM_MINMAX)
            processed_frame = np.uint8(processed_frame)

            # Convert back to BGR
            processed_frame = cv2.cvtColor(processed_frame, cv2.COLOR_GRAY2BGR)

            # Add title for high-pass filter
            cv2.putText(processed_frame, 'Gaussian High Pass Filter', (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, .5, (0, 0, 255), 2)

        elif process_type == "fourier_band":
            # Apply Gaussian Band Pass Filter
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            # Create Gaussian Low-Pass Kernel
            low_pass_size = 15
            low_pass_sigma = 15
            low_pass_kernel = gaussian_kernel(low_pass_size, low_pass_sigma)

            # Create Gaussian High-Pass Kernel
            high_pass_size = 15
            high_pass_sigma = 5
            high_pass_kernel = gaussian_kernel(high_pass_size, high_pass_sigma)

            # Apply the low-pass filter
            low_passed = cv2.filter2D(gray, -1, low_pass_kernel)

            # Apply the high-pass filter
            high_passed = cv2.filter2D(gray, -1, high_pass_kernel)

            # Combine the results to create a band-pass effect
            processed_frame = cv2.addWeighted(low_passed, 1, high_passed, -1, 0)

            # Normalize the result
            processed_frame = cv2.normalize(processed_frame, None, 0, 255, cv2.NORM_MINMAX)
            processed_frame = np.uint8(processed_frame)
            processed_frame = cv2.cvtColor(processed_frame, cv2.COLOR_GRAY2BGR)

            cv2.putText(processed_frame, 'Gaussian Band Pass Filter', (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, .5, (0, 0, 255), 2)

        elif process_type == "template_matching_s_number":
            # Check if templates are provided
            if templates is not None:
                # Perform template matching for s-number
                processed_frame = match_template(frame, templates['s_number'])
                cv2.putText(processed_frame, 'Student number', (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, .5, (0, 0, 255), 2)

        elif process_type == "template_matching_logo":
            # Check if templates are provided
            if templates is not None:
                # Perform template matching for logo
                processed_frame = match_template(frame, templates['logo'])
                cv2.putText(processed_frame, 'UT Logo', (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, .5, (0, 0, 255), 2)

        elif process_type == "template_matching_photo":
            # Check if templates are provided
            if templates is not None:
                # Perform template matching for photo
                processed_frame = match_template(frame, templates['photo'])
                cv2.putText(processed_frame, 'ID photo', (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, .5, (0, 0, 255), 2)

        elif process_type == "optical flow":
            # Initialize previous grayscale frame if not available
            if not hasattr(apply_processing, "prev_gray"):
                apply_processing.prev_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            # Convert current frame to grayscale
            curr_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            # Check if sizes match before calculating optical flow
            if apply_processing.prev_gray.shape == curr_gray.shape:
                # Calculate optical flow (Farneback method)
                flow = cv2.calcOpticalFlowFarneback(apply_processing.prev_gray, curr_gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)

                # Draw optical flow arrows
                processed_frame = draw_optical_flow_arrows(frame, flow)

                # Update previous grayscale image for the next frame
                apply_processing.prev_gray = curr_gray

        elif process_type == "laplacian":
            # Apply Laplacian filter
            processed_frame = cv2.Laplacian(frame, cv2.CV_64F)
            processed_frame = cv2.convertScaleAbs(processed_frame)
            cv2.putText(processed_frame, 'Laplacian Filter', (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, .5, (255, 255, 255), 2)

        elif process_type == "color_map":
            # Apply a color map
            processed_frame = cv2.applyColorMap(frame, cv2.COLORMAP_JET)
            cv2.putText(processed_frame, 'Color Map', (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, .5, (255, 255, 255), 2)

        elif process_type == "negative":
            # Invert colors
            processed_frame = cv2.bitwise_not(frame)
            cv2.putText(processed_frame, 'Negative', (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, .5, (255, 255, 255), 2)

        elif process_type == "adaptive_threshold":
            # Apply adaptive threshold
            gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            processed_frame = cv2.adaptiveThreshold(gray_frame, 255,
                                                    cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                                    cv2.THRESH_BINARY, 11, 2)
            cv2.putText(processed_frame, 'Adaptive Threshold', (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, .5, (0, 0, 255), 2)

        elif process_type == "rotation":
            # Rotate the frame by a specified angle (e.g., 45 degrees)
            angle = 45
            center = (frame.shape[1] // 2, frame.shape[0] // 2)
            rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
            processed_frame = cv2.warpAffine(frame, rotation_matrix, (frame.shape[1], frame.shape[0]))
            cv2.putText(processed_frame, 'Rotation', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, .5, (0, 0, 255), 2)

        elif process_type == "affine":
            # Apply affine transformation
            rows, cols = frame.shape[:2]
            points1 = np.float32([[50, 50], [200, 50], [50, 200]])
            points2 = np.float32([[10, 100], [200, 50], [100, 250]])
            affine_matrix = cv2.getAffineTransform(points1, points2)
            processed_frame = cv2.warpAffine(frame, affine_matrix, (cols, rows))
            cv2.putText(processed_frame, 'Affine Transform', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, .5, (0, 0, 255), 2)

    return processed_frame

In [33]:
## vedio frame process, time management operations
def main(input_video_file: str, output_video_file: str) -> None:
    # OpenCV video objects to work with
    cap = cv2.VideoCapture(input_video_file)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    fps = int(round(cap.get(cv2.CAP_PROP_FPS)))
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Saving output video as .mp4
    out = cv2.VideoWriter(output_video_file, fourcc, fps, (frame_width, frame_height))

    frame_count = 0  # Counter for the number of frames processed

    id_card_region = None  # Initialize ID card region variable

    # While loop where the real work happens
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break  # Break if frame read fails

        frame_count += 1  # Increment frame count

        # Calculate current time in milliseconds
        current_time_ms = frame_count * 1000 // fps  # Convert frame count to milliseconds

        # Detect ID card region only once for relevant sections
        if 20000 <= current_time_ms < 40000:
            frame, id_card_region = id_card_detection(frame)

        # Apply different effects based on the timestamp
        if between(cap, 0, 2500):
            frame = apply_processing(cap, frame, 0, 2500, "blur", templates)
        elif between(cap, 2500, 5000):
            frame = apply_processing(cap, frame, 2500, 5000, "sharpen", templates)
        elif between(cap, 5000, 7500):
            frame = apply_processing(cap, frame, 5000, 7500, "sobel", templates)
        elif between(cap, 7500, 10000):
            frame = apply_processing(cap, frame, 7500, 10000, "canny", templates)
        elif between(cap, 10000, 12500):
            frame = apply_processing(cap, frame, 10000, 12500, "dft", templates)
        elif between(cap, 12500, 15000):
            frame = apply_processing(cap, frame, 12500, 15000, "fourier_low", templates)
        elif between(cap, 15000, 17500):
            frame = apply_processing(cap, frame, 15000, 17500, "fourier_high", templates)
        elif between(cap, 17500, 20000):
            frame = apply_processing(cap, frame, 17500, 20000, "fourier_band", templates)
        elif 20000 <= current_time_ms < 25000 and id_card_region is not None:
            frame = apply_processing(cap, frame, 20000, 25000, "template_matching_s_number", templates)
        elif 25000 <= current_time_ms < 30000 and id_card_region is not None:
            frame = apply_processing(cap, frame, 25000, 30000, "template_matching_logo", templates)
        elif 30000 <= current_time_ms < 35000 and id_card_region is not None:
            frame = apply_processing(cap, frame, 30000, 35000, "template_matching_photo", templates)
        elif 35000 <= current_time_ms < 40000 and id_card_region is not None:
            frame = apply_processing(cap, frame, 35000, 40000, "optical flow", templates)
        # Apply different effects based on the timestamp
        elif between(cap, 40000, 43000):
            frame = apply_processing(cap, frame, 40000, 43000, "affine", templates)
        elif between(cap, 43000, 46000):
            frame = apply_processing(cap, frame, 43000, 46000, "color_map", templates)
        elif between(cap, 46000, 49000):
            frame = apply_processing(cap, frame, 46000, 49000, "negative", templates)
        elif between(cap, 49000, 52000):
            frame = apply_processing(cap, frame, 49000, 52000, "laplacian", templates)
        elif between(cap, 52000, 55000):
            frame = apply_processing(cap, frame, 52000, 55000, "adaptive_threshold", templates)
        elif between(cap, 55000, 58000):
            frame = apply_processing(cap, frame, 55000, 58000, "rotation", templates)

        # Write processed frame to output
        out.write(frame)

        # Display every 30th frame using cv2_imshow
        if frame_count % 30 == 0:
            cv2_imshow(frame)

        # Optional: Press Q on keyboard to exit
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break

    # When everything is done, release the video capture and writing object
    cap.release()
    out.release()



# Template images input and sizing

In [34]:
# Load templates for s-number, logo, and unique pattern
template_s_number = cv2.imread('/content/drive/MyDrive/Image processing assignments /Ferdous_assignment2/S_number_200.png', cv2.IMREAD_GRAYSCALE)
# Set a desired width for the resized template
desired_width_snumber = 150
aspect_ratio_snumber = template_s_number.shape[1] / template_s_number.shape[0]
desired_height_snumber = int(desired_width_snumber / aspect_ratio_snumber)
resized_template_snumber = cv2.resize(template_s_number, (desired_width_snumber, desired_height_snumber))

# Load templates for logo
template_logo = cv2.imread('/content/drive/MyDrive/Image processing assignments /Ferdous_assignment2/logo_resized.jpg', cv2.IMREAD_GRAYSCALE)
desired_width_logo = 350
aspect_ratio_logo = template_logo.shape[1] / template_logo.shape[0]
desired_height_logo = int(desired_width_logo / aspect_ratio_logo)
resized_template_logo = cv2.resize(template_logo, (desired_width_logo, desired_height_logo))


# Load templates for photo
template_photo = cv2.imread('/content/drive/MyDrive/Image processing assignments /Ferdous_assignment2/photo.png', cv2.IMREAD_GRAYSCALE)
desired_width_photo = 175
aspect_ratio_photo = template_photo.shape[1] / template_photo.shape[0]
desired_height_photo = int(desired_width_photo / aspect_ratio_photo)
resized_template_photo = cv2.resize(template_photo, (desired_width_photo, desired_height_photo))

# Assuming `templates` is defined as:
templates = {
    's_number': resized_template_snumber,
    'logo': resized_template_logo,
    'photo': resized_template_photo
}


# Input and Output

In [35]:
# Define the input and output file paths
input_video_file = '/content/drive/MyDrive/Image processing assignments /Ferdous_assignment2/Ferdous_1_60sec.mp4'  # Input video path
output_video_file = '/content/drive/MyDrive/Image processing assignments /Ferdous_assignment2/output_video_1_60.mp4'  # Output video path

main(input_video_file, output_video_file)

Output hidden; open in https://colab.research.google.com to view.